# UI

> A chat interface UI demo

In [ ]:
#| export
from fasthtml.common import *
from fhbasecoat.utils import *
from fhbasecoat.common import *
from fhbasecoat.interactive import *
from fhbasecoat.chat import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from itertools import product
from enum import Enum, auto


In [ ]:
app = FastHTML(exts="ws", session_cookie="mysession")
rt = app.route

In [ ]:
# srv.stop()

In [ ]:
srv = JupyUvi(app)

In [ ]:
from pathlib import Path

In [ ]:
! ls ~/git/fhbasecoat/fhbasecoat

__init__.py  _modidx.py  common.py	 utils.py
__pycache__  chat.py	 interactive.py


In [ ]:
common_components = Path("~/git/fhbasecoat/fhbasecoat/common.py").expanduser().read_text()
interactive_components = Path("~/git/fhbasecoat/fhbasecoat/interactive.py").expanduser().read_text()

Note: whenever you answer questions from now you should refer to $`[common_components, interactive_components]`.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Got it! I can see you're working with FastHTML and have loaded some component code from `fhbasecoat`. I'll reference the `common_components` and `interactive_components` code when answering your questions.

What would you like to explore or work on with these components?

# Code

In [ ]:
ButtonGroup??


```python
def ButtonGroup(*args, cls="", **kwargs):
    return Div(*args, role="group", cls=f"button-group {cls}", **kwargs)
```

**File:** `~/git/fhbasecoat/fhbasecoat/common.py`

In [ ]:
def SearchBar():
    return fh.Button(
            Div(
                Div(
                    Icon("search", cls="size-4"),
                    Span("Search Documentation", cls="text-sm"),
                    cls="flex gap-2 items-center",
                ),
                Kbd("⌘ K", cls="text-2xs"),
                cls="flex items-center px-3 py-2 justify-between",
            ),
            cls="m-1 hover:cursor-pointer bg-background rounded-lg",
        )

In [ ]:
def DemoSidebar():
    return Sidebar(
        SearchBar(),
        Group(
            "Platform",
            SidebarCollapsable(IconTitle("Playground", icon="square-terminal"), ["History", "Starred", "Settings"]),
            SidebarCollapsable(IconTitle("Models", icon="bot"), ["Genesis", "Explorer", "Quantum"]),
            SidebarCollapsable(IconTitle("Documentation", icon="book-open"), ["Introduction", "Get Started", "Tutorials", "Changelog"]),
            SidebarCollapsable(IconTitle("Settings", icon="settings-2"), ["General", "Team", "Billing", "Limits"]),
        ),
        SidebarGroup("Projects", ["Design Engineering", "Sales & Marketing", "Travel"], icon_list=["frame", "chart-pie", "map"]),
        footer=Div(
                Dropdown(
                    DropdownItem("Upgrade to Pro"),
                    Separator(),
                    DropdownItem("Account"),
                    trigger_btn=DropdownTriggerButton(
                        Div(
                            Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), Span("Taya", cls="pl-2"),
                            cls="flex items-center w-full",
                        ),
                        Icon("chevrons-up-down"), did="sidebar-dropdown", 
                        cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                        style="width: calc(100% - 0.5rem)"
                    ),
                    id="sidebar-dropdown",
                    side="right", align="end",
            ),
            cls="flex flex-col items-left w-full",
        ),
    )

pw(
    DemoSidebar()
)

Okay I like the way this semantic search looks. However, there seems to be some corner problems perhaps because I'm using Input. I don't think I need input, but A P("") doesn't have the same styling. Could you suggest an alternative that looks exactly the same as above without Input?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, the `Input` is giving you nice styling but you don't need the input functionality. You can replace it with a `Span` or `P` and apply the same classes that `Input` would have.

Try replacing this:

```python
Input(value="Semantic search", type="text", cls="pl-9 rounded-lg", disabled=True)
```

with:

```python
Span("Semantic search", cls="pl-9 rounded-lg py-2 px-3 border border-input bg-background")
```

The key is adding the border, padding, and background classes that inputs typically have. Does this give you the look you're after, or do you need to adjust the styling?

Can you fix everything I mean. We don't need absolute anymore if not doing input.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good point! Without the input trick, you can simplify to a flex layout. Here's the cleaner version:

```python
fh.Button(
    Div(
        Icon("search", cls="size-4"),
        Span("Semantic search"),
        cls="flex items-center gap-2 py-2 px-3 border border-input bg-background rounded-lg",
    ),
    cls="m-1 hover:cursor-pointer bg-background",
),
```

This removes the `relative`/`absolute` positioning and just uses `flex items-center gap-2` to put the icon and text side by side. Does that work for you?

I was just on a website where I could use ctrl + k hotkey to open search. Normally this opens chrome search but somehow this was overriden. How is that possible?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! Websites can override default browser keyboard shortcuts using JavaScript event listeners. They intercept the keypress before it reaches the browser's default handler.

The basic approach is to listen for keyboard events and call `preventDefault()` to stop the browser's default action. Here's a simple example:

```javascript
document.addEventListener('keydown', (e) => {
  if ((e.ctrlKey || e.metaKey) && e.key === 'k') {
    e.preventDefault();
    // Open your custom search here
  }
});
```

The key parts are:
- `e.ctrlKey` checks if Ctrl is pressed (or `e.metaKey` for Cmd on Mac)
- `e.key === 'k'` checks if the K key is pressed
- `e.preventDefault()` stops the browser's default behavior

Have you tried implementing something like this, or would you like to see how to integrate it with FastHTML components?

Looking at my semantic search part, I'm trying to get the search icon to the left of the placeholder, but they are overlapping currently.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, you have the right idea with the relative positioning, but the issue is that the `Button` with the icon is positioned absolutely, which places it on top of the `Input` field.

To fix the overlapping, you need to add padding to the left of the `Input` to make room for the icon button. Try adding `pl-10` or `pl-12` to your Input's class:

```python
Input(type="text", placeholder="Semantic search", cls="pl-10"),
```

This will push the placeholder text (and any typed text) to the right, giving space for the icon button on the left.

Does that solve the overlap issue for you?

Okay, that is better. How can I do items center when I have a relative div?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To center items vertically within a relative div, you can add flexbox utilities. Try adding `flex items-center` to your relative div's class:

```python
cls="relative flex items-center"
```

This will vertically center both the `Input` and the `Icon` within the container.

Does that give you the alignment you're looking for?

I'm pretty sure that doesn't work because the asolute thing is saying put this at top left, so flex is not really impacting it at all.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're exactly right! The `absolute` positioning on the Icon takes it out of the normal document flow, so flexbox won't affect it. The `left-1 top-1` is what's positioning it.

To vertically center the absolutely positioned icon, you could use `top-1/2` and `-translate-y-1/2` to center it vertically:

```python
Icon("search", cls="absolute left-3 top-1/2 -translate-y-1/2 size-4"),
```

This positions it 50% from the top, then translates it back up by 50% of its own height to center it.

Would you like to try that approach?

In [ ]:
def TopBar():
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb(["Chats", "Demo Chat"]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

pw(TopBar())

## Chat interface

In [ ]:
from lisette import *
chat = Chat(model="anthropic/claude-3-haiku-20240307")
chat

<lisette.core.Chat>

In [ ]:
r = chat("testing", stream=True)

In [ ]:
list_test = list(r)

In [ ]:
for r in chat("Testing2", stream=True):
    delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
    if delta and delta.content:
        print(delta.content, end='', flush=True)
final_response = r.choices[0].message.content

Okay, I

'm ready to assist

 you with

 whatever you need.

 Please let me know

 how

 I can help!

What is a good way to print all things, and also to get deltas when streaming, and then final thing afterwards?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code, the issue is that when streaming ends, the response structure changes. Here's a good pattern for handling streaming responses:

```python
for r in chat("Testing2", stream=True):
    delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
    if delta and delta.content:
        print(delta.content, end='', flush=True)
```

This checks if `delta` exists before accessing it. When the stream ends, you'll get a different structure without a `delta` attribute.

Want to also capture the final complete message? You could accumulate the chunks:

```python
chunks = []
for r in chat("Testing2", stream=True):
    delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
    if delta and delta.content:
        chunks.append(delta.content)
        print(delta.content, end='', flush=True)

final_message = ''.join(chunks)
```

Would you like to try this approach, or would you prefer a different way to handle the streaming?

In [ ]:
async for s in await r:
    print(s.choices[0].delta.content)

How do I get streaming content?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your error, the issue is that when streaming, the structure is different from a regular response. With streaming, you get `delta` instead of `message`.

Try this:

```python
async for s in await r:
    print(s.choices[0].delta.content)
```

The streaming response uses `delta` to send incremental chunks of the content as it's generated, rather than a complete `message` object.

Does that work for you?

How do I loop through r?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You need to `await` the coroutine first to get the async iterator. Try this:

```python
async for s in await r:
    print(s)
```

The issue is that `chat("testing", stream=True)` returns a coroutine, which you need to await to get the actual async iterator that you can loop through.

Does that work for you?

Hmm, I still don't get how I access the content at each step.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your earlier output, you can see that streaming responses have `delta` objects with `content` in them. So you'd access it like:

```python
s.choices[0].delta.content
```

However, note that the last item in the stream is a complete `ModelResponse` (not a `ModelResponseStream`), so it has `message` instead of `delta`. You might want to check which type you're dealing with.

Want to try accessing the content and see what happens?

In [ ]:
s.choices[0].delta

In [ ]:
# print(r.choices[0].message.content)

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, send):
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    r = chat(text, stream=True)
    output = r.choices[0].message.content
    await send(Div(ChatAssistant(output), id="chat-interface", hx_swap_oob="beforeend"))

In [ ]:
CardOutline??


```python
def CardOutline(*contents, cls="", **kwargs):
    return Div(*contents, cls=f"bg-background border border-border {cls}", **kwargs)
```

**File:** `~/git/fhbasecoat/fhbasecoat/common.py`

In [ ]:
#| export
def ChatMessage(content:str, msg_type="", cls="", rounding="", color=""): 
    return Div(
        Div(
            Div(
                P(msg_type, cls="ml-2"), cls=f"text-xs text-muted-background border-b border-border hover:cursor-pointer {rounding} rounded-b-none {color}"),
                Div(content, cls="px-4 py-3 whitespace-pre-wrap"),
        ),
        cls=f"border border-border text-sm {cls} {rounding}"
    )

def ChatPrompt(content:str):
    return ChatMessage(content, msg_type="Prompt", cls="bg-card w-[85%]", rounding="rounded-2xl rounded-tl-sm", color="bg-rose-700/70")

def ChatAssistant(content:str):
    return ChatMessage(content, msg_type="Assistant", cls="bg-background w-[90%] ml-auto mb-2", rounding="rounded-2xl", color="bg-rose-700/70")


In [ ]:
#| export
def ChatInterface(*contents, chat_input=None, cls="", id=None):
    return Div(
        Div(
            *contents,
            cls="bg-background border-l border-r flex flex-col flex-1 mx-1 p-3 gap-2",
            id=id,
        ),
        chat_input,
        cls=f"flex flex-col w-full lg:max-w-4xl mx-auto {cls}",
    )

In [ ]:
@rt
def demo_chat():
    return Div(
        DemoSidebar(),
        Div(
            TopBar(),
            ChatInterface(
                ChatPrompt("A test message sent by a user\n\nAnd new lines\nAre here."),
                ChatAssistant("A smaller response returned by a LLM. Great question. You are absolutely right!"),
                chat_input=ChatInput(
                    cls="m-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                    hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
                ),
                cls="flex-1",
                id="chat-interface",
            ),
            cls="h-full flex flex-col",
        ),
        cls="h-screen"
    )

## Things to add

- A llm backend integration that puts out prompt and assistant
- Backend websockets for streaming
- Token count in top panel using tiktoken (And token (i) hover showing cost)
- Click to edit prompt/assistant output
- LLM thinking mode button + display live thinking output
- Switch between prompt + Note
- Note Output
- Note output integration with 
- Python result using web-python interpreter
- Code output type
- Hotkeys for sidebar, submit, switch type, etc.

### Things I've learnt

When using flexboxes, a child has to have a parent with flex to be able to use it.
For dynamic styling you can do `w-full lg:max-w-4xl mx-auto` to have something dynamically resize from full to a larger size.

htmx things:
- hotkeys by doing `hx_trigger="keydown[ctrlKey && key=='Enter'] from:body`
- Reset form by doing `hx_on__after_request="this.reset()"`
- Button auto get `type="submit"` when inside a form.
- To allow trigger to work with clicks as well you can do `click from:button, keydown...`
- To disable during the request you can do `hx_disabled_elt="find button, find textarea"`
- To clear textarea you can do `hx_on__before_request="this.reset()"`